# Importing Libraries
in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [29]:
# importing all the necessary libraries
import pandas as pd

from numpy import mean
import numpy as np
import time

# step 1: preprocessing
from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)

from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)

from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_classif, VarianceThreshold

# step 2: data division
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score, GridSearchCV, ParameterGrid # to divide the code into train/test using a specific percentage or with/without replacement

# step 3: model
from sklearn.tree import DecisionTreeClassifier                                                        
from sklearn.naive_bayes import GaussianNB                                                              
from sklearn.neighbors import KNeighborsClassifier                                                       
from sklearn.ensemble import BaggingClassifier, VotingClassifier, ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier
import lightgbm as lgb 
import xgboost as xgb
from catboost import CatBoostClassifier 

# step 4: displaying accuracy
from sklearn.metrics import roc_auc_score, accuracy_score # to display the accuracy of our tree

# step 5: warning filter
import warnings
warnings.filterwarnings('ignore')

In [30]:
# use this block to install any libraries not on the system
# !pip install pandas
# !pip install sklearn
# python -m pip install scikit-learn lightgbm xgboost catboost

# Data Loading
data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [31]:
# lets load the training data set
train_data = pd.read_csv(r"/kaggle/input/imlchallenger1/train_set.csv")

# lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y
train_data.head() 

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [32]:
# lets load the test data
test_data = pd.read_csv(r"/kaggle/input/imlchallenger1/test_set.csv")

# check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data
test_data.head() 

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78
0,300001,79.000000,17.122318,0,0,1,170.200000,1,700,0,...,0.070000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,300002,38.000000,43.693579,0,0,1,165.100000,1,814,0,...,0.050000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,300003,36.064225,23.998944,0,0,1,167.086735,1,662,0,...,0.006948,0.006948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,300004,61.846764,31.693449,0,3,1,182.355708,2,862,0,...,0.062613,0.033153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,300005,71.591991,20.086147,1,0,1,166.704917,2,335,0,...,0.014854,0.004854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Preprocessing
before we start processing this data and using algorithms, we will fix this data first, this is called data preprocessing

## Conversion of Categorical to Numerical
First we will convert categorical data to numerical data by doing one hot encoding, which turns it into binary variables

In [33]:
# one hot encoding - display it
pd.get_dummies(train_data) # this line will convert the train_data to one hot encoding but it will only display the result and not save it

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246117,246118,65.149110,33.357948,0,0,1,156.317941,1,711,0,...,0.027152,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
246118,246119,48.000000,46.736176,0,0,1,157.000000,1,594,2,...,0.560000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,1
246119,246120,57.472080,41.854115,1,0,0,189.868698,2,455,0,...,0.020601,0.0,0.0,0.587987,0.0,0.412013,0.0,0.0,0.412013,0
246120,246121,66.000000,23.738662,1,0,1,168.000000,2,609,0,...,-1.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.000000,0


In [34]:
# we can see that there is no change in the number of columns meaning there is no categorical data. but for the sake of running the program. we must perform the preprocessing therefore we shall re-run the one hot encoding and save it somewhere
train_data_processed = pd.get_dummies(train_data)

# now we shall do the same on the test data so that we maintain the rules over all data
test_data_processed = pd.get_dummies(test_data)

## Data Splitting - festures and targets
the data in train_data set is of x1 - x78 columns (79 variables) and one target variable (Y). we must split that data so that we can perform data preprocessing on the features variables (will be referred to as X).

In [35]:
# so in X, it is ALL the columns EXCEPT the last column known as 'Y' (we can confirm this using the train_data.head() we did earlier) so we must get all columns and DROP only the 'y' column
X = train_data_processed.drop(columns=['Y'])
X # lets display X and see what it is now

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.110000,0.040000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.100292,0.033431,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.020000,0.010000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.108249,0.039363,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.164645,0.069242,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246117,246118,65.149110,33.357948,0,0,1,156.317941,1,711,0,...,0.088610,0.027152,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
246118,246119,48.000000,46.736176,0,0,1,157.000000,1,594,2,...,-1.000000,0.560000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
246119,246120,57.472080,41.854115,1,0,0,189.868698,2,455,0,...,0.032961,0.020601,0.0,0.0,0.587987,0.0,0.412013,0.0,0.0,0.412013
246120,246121,66.000000,23.738662,1,0,1,168.000000,2,609,0,...,0.020000,-1.000000,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0,0.000000


In [36]:
# so as per our X output, we can see that number of columns in train_data is 79 and number of columns in X is 78 meaning we have successfully performed our removal of target variable
# now to get the target variable alone, we can just get it alone,
Y = train_data_processed['Y']
Y # lets see what it is
# as per our Y output, we can see it is of one column and 246k rows which means we have successfully extracted the target variable column

0         0
1         0
2         0
3         0
4         0
         ..
246117    0
246118    1
246119    0
246120    0
246121    0
Name: Y, Length: 246122, dtype: int64

## Data Imputation 
many cells in our data may be empty - we must fill these cells with data. we have multiple options to deal with them:
- we remove the entire rows (Case 1)
- we fill the cells with the average of the column (Case 2)
- we fill the cells based on KNN imputation (nearest neighbour) (Case 3)

In [37]:
# Average Mean Imputation
# ----------------------------- case -----------------------------
# this will fill all the empty spaces using the average of all the spaces
imputer = SimpleImputer(strategy='mean')

In [38]:
# KNN Imputation
# ----------------------------- case -----------------------------
# this fills them in using k-nearest neighbours of all the spaces
# imputer = KNNImputer(n_neighbors=7)

In [39]:
X = imputer.fit_transform(X)                                        # fill them in X
test_data_processed = imputer.transform(test_data_processed)    # fill them in test data

## Data Scaling
some columns may be very large then other columns when compared. it would not affect at the moment as we are using decision trees, but to maintain a fair enviroment, we shall perform scaling on every run.
there are two types of scaling: 
- min max scaling (also known as normalization)
- standardisation (z-score normalization)
- max abs scaler
- robust scaler
- normalizer

In [40]:
# ----------------------------- case  -----------------------------
# in this case we shall perform min max scaling. to do that, we must use our MinMaxScaler that we have imported above
scaler = MinMaxScaler()
# now we must use this scaler to scale X
scaler.fit_transform(X)

array([[0.00000000e+00, 9.72602740e-01, 3.63856188e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.06304216e-06, 9.09209364e-01, 3.15807703e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.12608432e-06, 4.65753425e-01, 2.44038356e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.99991874e-01, 5.68110690e-01, 5.09895343e-01, ...,
        0.00000000e+00, 0.00000000e+00, 4.12013395e-01],
       [9.99995937e-01, 6.84931507e-01, 1.67901180e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 4.69282739e-01, 3.38096342e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [41]:
# ----------------------------- case -----------------------------
# scaler = MaxAbsScaler()
# # now we must use this scaler to scale X
# scaler.fit_transform(X)

In [42]:
# our output shows us that every value in the array is between 0 and 1. thus lets save this value on X
X = scaler.fit_transform(X)

# now we must do the same on our test_data set
test_data_processed = scaler.transform(test_data_processed)

# Filters
there are two types of filters to filter out columns/features:
- variance filter (a column which has same values throughout the column like all are sunny)
- correlation filter (two columns which are same like weight in kg and weight in pounds)

In [43]:
print("X : ", X.shape)
print("test data : ", test_data_processed.shape)

X :  (246122, 78)
test data :  (105482, 78)


In [44]:
# variance filter
# ----------------------------- case  -----------------------------
# variance_filter = VarianceThreshold(threshold=0.001)  # Adjust the threshold if needed
# X = variance_filter.fit_transform(X)
# test_data_processed = variance_filter.fit_transform(test_data_processed)
X.shape

(246122, 78)

In [45]:
test_data_processed.shape

(105482, 78)

In [46]:
# # correlation filter
# # ----------------------------- case  -----------------------------
# corr_matrix = pd.DataFrame(X).corr().abs()
# upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]
# X = pd.DataFrame(X).drop(columns=to_drop)
# test_data_processed = pd.DataFrame(test_data_processed).drop(columns=to_drop)
X.shape

(246122, 78)

In [47]:
test_data_processed.shape

(105482, 78)

## Data Splitting - train and validate
now our test_data set is of rows with NO target variable whereas the train_data set is WITH target variable.
our rules in machine learning is that we must train half or 70% of the data and then we must check its accuracy using the remaining half or 30% of the data - we can only check accuracy IF we have the answers i.e. the target variable. 
So, what we need to do is, is split the train_data set into 2, by a 70% and 30% ratio. we train the model using the 70% and then test the model using the 30% and then use that model to predict the test_data set.

In [48]:
# holdout method
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3, random_state=2)

# functions

In [49]:
def fbselection(direction, sample_model, features):
    selection = SequentialFeatureSelector(sample_model, direction=direction, n_features_to_select=features, scoring='roc_auc')
    return modelSelector(sample_model, selection)

def modelSelector(sample_model, selection):
    trainX = selection.fit_transform(trainX, trainY)
    testX = selection.transform(testX)                                  # Ensure the test set is transformed similarly
    test_data_processed = selection.transform(test_data_processed)      # test data is also transformed
    X = selection.transform(X)                                          # full data transforming
    return sample_model

def kbest(sample_model, features):
    selection = SelectKBest(score_func=f_classif, k=features)
    return modelSelector(sample_model, selection)

In [50]:
def featureImportance(sample_model, features, X, trainX, trainY, testX, test_data_processed):
    print("fitting")
    
    # fit the model
    sample_model.fit(trainX, trainY)

    print("extracting features")

    # extract all the feature names from data
    importances = sample_model.feature_importances_
    feature_names = train_data_processed.drop(columns=['Y']).columns
    print(feature_names)

    # sort with respect to importance
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # extract the top ones
    top_features = feature_importance_df['Feature'].head(features).values
    print(top_features)

    # change all data according to the top ones we have selected
    trainX = pd.DataFrame(trainX, columns=feature_names)[top_features]
    testX = pd.DataFrame(testX, columns=feature_names)[top_features]
    X = pd.DataFrame(X, columns=feature_names)[top_features]
    test_data_processed = pd.DataFrame(test_data_processed, columns=feature_names)[top_features]

    print("features extracted")
    
    # retrain the model
    sample_model.fit(trainX, trainY)

    print("features trained")
    
    return sample_model, X, trainX, trainY, testX, test_data_processed

## model intialization
here model is intialized

In [51]:
### SAMPLE ###
# -------------------- case X (add the case number here) --------------------
# # intialize models here as model_1, model_2, perform feature selection and feature importance BEFORE they are inserted in stacking
# model_1 = 
# model_2 = 
# # intialize estimators here
# estimators = [('model_1', model_1), ('model_2', model_2)]
# # intialize stacking
# model = StackingClassifier(estimators=estimators, final_estimator=model_name)

In [52]:
# # -------------------- case 191d --------------------
# # intialize models here as model_1, model_2, perform feature selection and feature importance BEFORE they are inserted in stacking
# model_3 = xgb.XGBClassifier(n_estimators=2000 ,learning_rate= 0.03, max_depth = 4, random_state  = 42, device = "cuda")
# model_3, X, trainX, trainY, testX, test_data_processed = featureImportance(model_3, 40, X, trainX, trainY, testX, test_data_processed)
# model_2 = lgb.LGBMClassifier(learning_rate=0.02, max_depth=2, n_estimators=4000 , device='gpu')
# model_1 = BaggingClassifier(estimator= model_2, n_estimators=100, verbose=2, n_jobs=-1)
# # intialize estimators here
# estimators = [('model_1', model_1), ('model_2', model_2), ('model_3', model_3)]
# # intialize stacking
# model = StackingClassifier(estimators=estimators, final_estimator=model_2)

In [53]:
# -------------------- case  --------------------
# intialize models here as model_1, model_2, perform feature selection and feature importance BEFORE they are inserted in stacking
model_1 = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=15, max_features=60, min_samples_leaf=80)
model_2 = BaggingClassifier(estimator=model_1, n_estimators=50, verbose=2)
# intialize estimators here
estimators = [('model_1', model_1), ('model_2', model_2)]
# intialize stacking
model = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier(), verbose=2)

In [54]:
# feature importance
model_4, X, trainX, trainY, testX, test_data_processed = featureImportance( xgb.XGBClassifier(), 45, X, trainX, trainY, testX, test_data_processed )

fitting
extracting features
Index(['RecordId', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20',
       'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30',
       'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40',
       'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50',
       'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60',
       'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70',
       'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')
['X26' 'X69' 'X74' 'X18' 'X73' 'X75' 'X28' 'X20' 'X6' 'X17' 'X58' 'X44'
 'X42' 'X15' 'X14' 'X70' 'X78' 'X59' 'X36' 'X38' 'X68' 'X52' 'X40' 'X39'
 'X35' 'X45' 'X66' 'X24' 'X23' 'X34' 'X11' 'X31' 'X37' 'X63' 'X43' 'X32'
 'X21' 'X72' 'X49' 'X60' 'X8' 'X77' 'X3' 'X10' 'X62']
features extracted
features trained


In [55]:
print("X shape -> ", X.shape)
print("trainX shape -> ", trainX.shape)
print("testX shape -> ", testX.shape)
print("test_data_processed shape -> ", test_data_processed.shape)

X shape ->  (246122, 45)
trainX shape ->  (172285, 45)
testX shape ->  (73837, 45)
test_data_processed shape ->  (105482, 45)


## Bagging intialization
here we will introduce and intialize bagging

In [56]:
# model = BaggingClassifier(estimator=model, n_estimators=50, verbose=2)
# -- 

## model running
here we run the model

In [57]:
# fit the model
model.fit(trainX, trainY)

Building estimator 1 of 50 for this parallel run (total 50)...
Building estimator 2 of 50 for this parallel run (total 50)...
Building estimator 3 of 50 for this parallel run (total 50)...
Building estimator 4 of 50 for this parallel run (total 50)...
Building estimator 5 of 50 for this parallel run (total 50)...
Building estimator 6 of 50 for this parallel run (total 50)...
Building estimator 7 of 50 for this parallel run (total 50)...
Building estimator 8 of 50 for this parallel run (total 50)...
Building estimator 9 of 50 for this parallel run (total 50)...
Building estimator 10 of 50 for this parallel run (total 50)...
Building estimator 11 of 50 for this parallel run (total 50)...
Building estimator 12 of 50 for this parallel run (total 50)...
Building estimator 13 of 50 for this parallel run (total 50)...
Building estimator 14 of 50 for this parallel run (total 50)...
Building estimator 15 of 50 for this parallel run (total 50)...
Building estimator 16 of 50 for this parallel run

StackingClassifier(estimators=[('model_1',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=5,
                                                       max_features=60,
                                                       min_samples_leaf=80,
                                                       min_samples_split=15)),
                               ('model_2',
                                BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                   max_depth=5,
                                                                                   max_features=60,
                                                                                   min_samples_leaf=80,
                                                                                   min_samples_split=15),
                                                  n_estimators=50,
                                                  verbose=2))],
                   final_estimator=DecisionTreeClassifier(), verbose=2)

In [58]:
# predict using this model
y_pred = model.predict(testX)

In [59]:
# display the accuracy of this prediction
accuracy = accuracy_score(testY, y_pred)
print("model accuracy = ", accuracy, "   ")

# now lets calculate the ROC AUC score according to this prediction
roc_score = roc_auc_score(testY, y_pred)
print("roc score = ", roc_score, "   ")

model accuracy =  0.9948941587550957    
roc score =  0.5383070227984298    


## predict for test dataset
fit the model and predict for test dataset

In [60]:
model.fit(X, Y)

Building estimator 1 of 50 for this parallel run (total 50)...
Building estimator 2 of 50 for this parallel run (total 50)...
Building estimator 3 of 50 for this parallel run (total 50)...
Building estimator 4 of 50 for this parallel run (total 50)...
Building estimator 5 of 50 for this parallel run (total 50)...
Building estimator 6 of 50 for this parallel run (total 50)...
Building estimator 7 of 50 for this parallel run (total 50)...
Building estimator 8 of 50 for this parallel run (total 50)...
Building estimator 9 of 50 for this parallel run (total 50)...
Building estimator 10 of 50 for this parallel run (total 50)...
Building estimator 11 of 50 for this parallel run (total 50)...
Building estimator 12 of 50 for this parallel run (total 50)...
Building estimator 13 of 50 for this parallel run (total 50)...
Building estimator 14 of 50 for this parallel run (total 50)...
Building estimator 15 of 50 for this parallel run (total 50)...
Building estimator 16 of 50 for this parallel run

StackingClassifier(estimators=[('model_1',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=5,
                                                       max_features=60,
                                                       min_samples_leaf=80,
                                                       min_samples_split=15)),
                               ('model_2',
                                BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                   max_depth=5,
                                                                                   max_features=60,
                                                                                   min_samples_leaf=80,
                                                                                   min_samples_split=15),
                                                  n_estimators=50,
                                                  verbose=2))],
                   final_estimator=DecisionTreeClassifier(), verbose=2)

In [61]:
test_prediction = model.predict_proba(test_data_processed)

test_prediction=test_prediction[:, 1]

print(test_prediction)

[0. 0. 0. ... 0. 0. 0.]


## write into csv
now we write the predictions into the csv file

In [63]:
sample_data = pd.read_csv(r"/kaggle/input/sample-sub/sample_submission.csv")

sample_data['Y'] = test_prediction
sample_data

sample_data.to_csv(r"/kaggle/working/stacking1.csv", index=False)
sample_data

,RecordId,Y
0,300001,0.0
1,300002,0.0
2,300003,0.0
3,300004,0.0
4,300005,0.0
...,...,...
105477,405478,0.0
105478,405479,0.0
105479,405480,0.0
105480,405481,0.0


In [64]:
model

StackingClassifier(estimators=[('model_1',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=5,
                                                       max_features=60,
                                                       min_samples_leaf=80,
                                                       min_samples_split=15)),
                               ('model_2',
                                BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                   max_depth=5,
                                                                                   max_features=60,
                                                                                   min_samples_leaf=80,
                                                                                   min_samples_split=15),
                                                  n_estimators=50,
                                                  verbose=2))],
                   final_estimator=DecisionTreeClassifier(), verbose=2)